<a href="https://colab.research.google.com/github/StratagemGIS/notebooks/blob/main/data_processing/86_unsafe_intersections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vaasudevan Srinivasan 🧑🏻‍💻  
StratagemGIS Solutions

In [ ]:
import geopandas as gpd
import pooch

In [ ]:
nbrn_file = pooch.retrieve(
    'https://github.com/StratagemGIS/datasets/raw/refs/heads/main/vector/geonb_nbrn-rrnb_shp.zip',
    known_hash='90d36a48b0a53c6320890d1a3bcaee9f9b98d6ad734f910ac6998b01d974b5d5'
)

!unzip -qq -o Traffic_Accidents_102100.zip
accidents_file = 'Traffic_Accidents_102100'

In [ ]:
accidents = (
    gpd.read_file(accidents_file)
    .query('Year_ != 0')
    .reset_index(drop=True)
    .to_crs('EPSG:2953')
)

nbrn = (
    gpd.read_file(
        nbrn_file,
        layer='geonb_nbrn-rrnb_road-route',
        engine='pyogrio',
    )
    .dropna(subset=['STREETNAME'])
    .query('PLACE_NAME == "Fredericton"')
    .query('STREETNAME != "Unnamed"')
    .dissolve(by='STREETNAME')
    .reset_index()
    .assign(
        STREETNAME=lambda df_: df_.STREETNAME.str.title(),
        road_geom=lambda df_: df_.geometry,
    )
)

In [ ]:
intersection_pts = (
    gpd.overlay(nbrn, nbrn, how='intersection', keep_geom_type=False)
    .assign(geom_type=lambda df_: df_.geometry.geom_type)
   .query('geom_type.isin(("Point", "MultiPoint"))')
   .explode()
   .reset_index(drop=True)
   .assign(
       Intersection=lambda df_: df_.apply(
           lambda row: tuple(sorted((row.STREETNAME_1, row.STREETNAME_2))), axis=1
        )
    )
   [['Intersection', 'geometry']]
)

In [ ]:
result = (
    gpd.sjoin_nearest(accidents, intersection_pts, max_distance=20)
    .groupby('Intersection', as_index=False)
    .agg({'OBJECTID_1': 'count'})
    .rename(columns={'OBJECTID_1': 'Accidents_Count'})
    .sort_values('Accidents_Count', ascending=False)
    .reset_index(drop=True)
)

In [ ]:
result.head(10)

,Intersection,Accidents_Count
0,"(Prospect Street, Regent Street)",302
1,"(Maple Street, Ring Road)",220
2,"(Prospect Street, Smythe Street)",218
3,"(Hanwell Road, Prospect Street)",214
4,"(Priestman Street, Regent Street)",208
5,"(Regent Street, Vanier Highway)",198
6,"(Brunswick Street, Regent Street)",196
7,"(Bishop Drive, Hanwell Road)",174
8,"(Beaverbrook Street, Waterloo Row)",150
9,"(Knowledge Park Drive, Regent Street)",148
